In [4]:
# https://docs.cohere.com/reference/generate

import cohere
import os

co = cohere.Client(os.environ['COHERE_API_KEY'])

prompt = """
From the following historical document text, please grab out the following items and return with the following format
[{{
  "name": "John Doe",
  "age": 30,
  "title": "Farmer"
}},
{{<ITEM 2>}}, {{Item 3>}}, ...]

Here is the text:
{text}"""


response = co.generate(
  prompt='From the following',
)
print(response)


SyntaxError: incomplete input (3923542006.py, line 8)

In [9]:
# pip install -U langchain-community cohere
## Use Langchain https://python.langchain.com/docs/integrations/llms/cohere
# https://api.python.langchain.com/en/latest/llms/langchain_community.llms.cohere.Cohere.html

#from langchain_community.llms import Cohere
# from langchain_core.prompts import PromptTemplate

from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

import json

model = OpenAI()

system_message = """
From the following historical document text, please grab out the following items and return with the following format
[{{
  "name": "John Doe",
  "age": 30,
  "title": "Farmer"
}},
{{<ITEM 2>}}, {{Item 3>}}, ...]

Return only the list of Python dictionary objects and nothing more. Make a dictionary entry for each person mentioned.

Here is the text:
{text}"""

text = "Tim Paulson was a farmer who lived in the 1800s. He was 30 years old and lived in the state of California. He had two kids, John and Jane. John was 12 and Jane was 10. They were both students."

prompt = PromptTemplate(input_variables=['text'], template=system_message)
full_prompt = prompt.format(text=text)

response_raw = model.predict(full_prompt)

response = json.loads(response_raw)

response




/opt/homebrew/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\n\n[{\n  "name": "Tim Paulson",\n  "age": 30,\n  "title": "Farmer"\n},\n{\n  "name": "John Paulson",\n  "age": 12,\n  "title": "Student"\n},\n{\n  "name": "Jane Paulson",\n  "age": 10,\n  "title": "Student"\n}]'